In [1]:
import sys
import os

# Add project root and parent directory to path
project_root = os.path.abspath("..")
parent_dir = os.path.abspath(os.path.join("..", ".."))

if project_root not in sys.path:
    sys.path.append(project_root)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


In [23]:
from sbmlParser.parser import ParseSBMLFile


In [24]:
sbml_file = "../data/euromix.sbml"  # Or your SBML file
model_data = ParseSBMLFile(sbml_file)

In [25]:
import sympy as sp
from sbml_rust_generator.models import SbmlModel

In [26]:
# Parse model into structured format
model = SbmlModel.from_dict(model_data)

In [32]:
species_list = list(model.species.keys())
species_map = {s_id: i for i, s_id in enumerate(species_list)}

In [35]:
# Create symbol dictionaries
sym_species = {s: sp.Symbol(s) for s in species_list}
sym_params = {p: sp.Symbol(p) for p in model.parameters.keys()}
sym_compartments = {c: sp.Symbol(c) for c in model.compartments.keys()}

In [39]:
# Combine all symbols for parser
all_syms = {**sym_species, **sym_params, **sym_compartments}

In [44]:
from sbml_rust_generator import SbmlExpressionParser, OdeSystemBuilder

parser = SbmlExpressionParser(all_syms, model_data.get("functions", {}))

In [45]:
# Build ODE system
ode_builder = OdeSystemBuilder(species_map, parser)
ode_system = ode_builder.build_ode_system(model_data["reactions"])

Parsing reactions...


In [46]:
ode_system

[1.0*FFat*QFat/(Fat*PCFat) - 1.0*FFat*QArt/Art,
 1.0*FRich*QRich/(PCRich*Rich) - 1.0*FRich*QArt/Art,
 1.0*FPoor*QPoor/(PCPoor*Poor) - 1.0*FPoor*QArt/Art,
 1.0*FLiver*QLiver/(Liver*PCLiver) - 1.0*QGut*kGut + 1.0*fub*Piecewise((Liver*Vmax*Piecewise((QLiver/Liver, Liver > 0), (0, True))/(Km*PCLiver + Piecewise((QLiver/Liver, Liver > 0), (0, True))), Michaelis > 0.5), (CLH*Piecewise((QLiver/Liver, Liver > 0), (0, True))/PCLiver, True)) - 1.0*FLiver*QArt/Art,
 -1.0*fub*Piecewise((Liver*Vmax*Piecewise((QLiver/Liver, Liver > 0), (0, True))/(Km*PCLiver + Piecewise((QLiver/Liver, Liver > 0), (0, True))), Michaelis > 0.5), (CLH*Piecewise((QLiver/Liver, Liver > 0), (0, True))/PCLiver, True)),
 1.0*QGut*kGut,
 1.0*FSkin_u*Piecewise((QSkin_u/Skin_u, Skin_u > 0), (0, True))/PCSkin - 1.0*f_su*Piecewise((QSkin_sc_u/Skin_sc_u, Skin_sc_u > 0), (0, True)) + 1.0*f_su*Piecewise((QSkin_u/Skin_u, Skin_u > 0), (0, True))/PCSkin_sc - 1.0*FSkin_u*QArt/Art,
 1.0*FSkin_e*Piecewise((QSkin_e/Skin_e, Skin_e > 0), (0

In [47]:
print(f"Built {len(ode_system)} ODEs")
print("\nFirst ODE:")
print(ode_system[0])

Built 14 ODEs

First ODE:
1.0*FFat*QFat/(Fat*PCFat) - 1.0*FFat*QArt/Art


In [54]:
# Cell 10: Compute sparse Jacobian
from sbml_rust_generator import JacobianBuilder

# Create Jacobian builder
jac_builder = JacobianBuilder(sym_species, species_list)

In [56]:
# Compute sparse Jacobian (only non-zero elements)
jac_elements, jac_indices = jac_builder.compute_sparse_jacobian(ode_system)

Computing Jacobian (sparse)...
Jacobian sparsity: 34/196 non-zero


In [60]:
from sbml_rust_generator import SymbolicOptimizer

optimizer = SymbolicOptimizer(optimization_level=2)



In [61]:
# Optimize ODE and Jacobian together
replacements, reduced_ode, reduced_jac = optimizer.optimize_combined(ode_system, jac_elements)

Optimizing ODE and Jacobian together...
Optimizing expressions with CSE...
DEBUG: Processing 74 CSE replacements for safe divisions...
DEBUG: Symbol x15 can be zero
DEBUG: x16 expression tree: 1/(Km*PCLiver + x15)
DEBUG: x16 expression type: <class 'sympy.core.power.Pow'>
DEBUG: x16 expression args: (Km*PCLiver + x15, -1)
DEBUG: x16 Pow#1: base=Km*PCLiver + x15, base_type=Add, exp=-1
DEBUG: x16 Pow#1: is base a Symbol? False
DEBUG: Found unsafe division in x16: Km*PCLiver + x15^-1 (contains zero symbol)
DEBUG: Rewriting x16 expression to make safe
DEBUG: Symbol x29 can be zero
DEBUG: Symbol x32 can be zero
DEBUG: Symbol x41 can be zero
DEBUG: Symbol x44 can be zero
DEBUG: Symbol x61 can be zero
DEBUG: Symbol x67 can be zero
DEBUG: Symbol x68 can be zero
DEBUG: Symbol x70 can be zero
DEBUG: Symbol x71 can be zero
DEBUG: Total zero-valued symbols found: 10
DEBUG: Zero symbols: {x15, x67, x61, x71, x41, x68, x44, x70, x32, x29}
CSE extracted 74 common subexpressions


In [63]:
from sbml_rust_generator.codegen import RustBlockGenerator

code_gen = RustBlockGenerator()

# Generate species extraction code
species_extract = code_gen.generate_species_extraction(species_map)

# Generate temporary variables (CSE)
temp_vars = code_gen.generate_temp_vars(replacements)

# Generate ODE derivatives
derivatives = code_gen.generate_derivatives(reduced_ode)

# Generate Jacobian-vector product
jacobian_code = code_gen.generate_jacobian(reduced_jac, jac_indices)

# Generate result vectors initialization
result_vectors = code_gen.generate_result_vectors_init(species_list)

# Generate result pushes
result_pushes = code_gen.generate_result_pushes(species_list)

# Generate HashMap inserts
hashmap_inserts = code_gen.generate_hashmap_inserts(species_list)

In [64]:
from sbml_rust_generator.codegen import RustTemplateManager

# Prepare code blocks dictionary
code_blocks = {
    "n_species": len(species_list),
    "species_extract": species_extract,
    "temp_vars": temp_vars,
    "rhs_block": derivatives,
    "jac_block": jacobian_code,
    "result_vectors_init": result_vectors,
    "loop_pushes": result_pushes,
    "initial_pushes": result_pushes,
    "hashmap_inserts": hashmap_inserts,
    "gut_idx": species_map.get("QGut", 0),  # Adjust for your model
}

# Assemble Rust file
template_manager = RustTemplateManager()
rust_code = template_manager.assemble_rust_file("my_model", code_blocks)



KeyError: 'species_fields'